In [ ]:
from pyasn1.type.univ import Null
#@title
# Decoding Lotto Info From Website

import pandas as pd
import requests
import json

from bs4 import BeautifulSoup
from urllib.request import urlopen

from google.colab import data_table
from vega_datasets import data

from tqdm import tqdm

# -------------------------------------------> 로또 데이터 웹 크롤링 ------------------------------------
  # 최고 회차 
def getMaxRoundNum() -> int:

    url = "https://dhlottery.co.kr/common.do?method=main"  
    html = requests.get(url)  #html = urlopen(url) 

    if html.text is not None:
      soup = BeautifulSoup(html.text, 'html.parser') #"lxml")
      tag = soup.find(name='strong', attrs={'id': 'lottoDrwNo'})
    else:
      raise Exception('Error getting data from {}'.format(url))
    
    return int(tag.text)

# 로또 Json URL : https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo= + (회차번호)
# 로또 회차 Json 구조
#     {"totSellamnt":103479378000,"returnValue":"success","drwNoDate":"2022-02-19",
#      "firstWinamnt":1811116822,"drwtNo6":45,"drwtNo4":39,"firstPrzwnerCo":14,"drwtNo5":43,
#      "bnusNo":31,"firstAccumamnt":25355635508,"drwNo":1003,"drwtNo2":4,"drwtNo3":29,"drwtNo1":1}

def getLottoWinInfo(minDrwNo, maxDrwNo):
    drwtNo1 = []
    drwtNo2 = []
    drwtNo3 = []
    drwtNo4 = []
    drwtNo5 = []
    drwtNo6 = []
    bnusNo = []
    drwNoDate = []
    roundNo = []

    for i in tqdm(range(minDrwNo, maxDrwNo+1)):
        req_url = "https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo=" + str(i)   
        req_lotto = requests.get(req_url)
        lottoNo = req_lotto.json()
        if not lottoNo["returnValue"]=="fail":
          drwtNo1.append(lottoNo["drwtNo1"])
          drwtNo2.append(lottoNo["drwtNo2"])
          drwtNo3.append(lottoNo["drwtNo3"])
          drwtNo4.append(lottoNo["drwtNo4"])
          drwtNo5.append(lottoNo["drwtNo5"])
          drwtNo6.append(lottoNo["drwtNo6"])
          bnusNo.append(lottoNo["bnusNo"])
          drwNoDate.append(lottoNo["drwNoDate"])
          roundNo.append(lottoNo["drwNo"])
        else:
          print("Parsing result is ", lottoNo["returnValue"], '\n')
          continue


    lotto_dict = {"RoundNo":roundNo, "Date":drwNoDate, "Num1":drwtNo1, "Num2":drwtNo2, "Num3":drwtNo3, 
                  "Num4":drwtNo4, "Num5":drwtNo5, "Num6":drwtNo6, "bnsNum": bnusNo}

    df_lotto = pd.DataFrame(lotto_dict)
    df_lotto.index = df_lotto.index + 1 

    return df_lotto


#--------------------------- 여기서부터는 구글드라이브 파일 처리---------------------------------------#
# Import Modules for Google Drive API
import gspread

from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

from google.colab import auth
from google.colab import drive, files
import os

# Helpful message to display if the CLIENT_SECRETS_FILE is missing.
MISSING_CLIENT_SECRETS_MESSAGE = """
WARNING: Please configure OAuth 2.0
To make this sample run you will need to populate the client_secrets.json file
found at:
  %s
with information from the APIs Console
<https://code.google.com/apis/console#access>.
""" 



def uploadDataFilesToMyDrive(filelocation):
  print ('Authenticating...')  
  auth.authenticate_user()

  if not os.path.isfile('{name}.csv'.format(name=filelocation)):
    print("'{0}' not found, upload .csv file".format(filelocation))
    uploaded = files.upload()

  if not os.path.isfile('{name}.json'.format(name=filelocation)):
    print("'{0}' not found, upload .json file".format(filelocation))
    uploaded = files.upload()
    
  drive_service = build('drive', 'v3')
  

  LOTTO_DATA_FILENAME='lottoWinInfo_{min}_{max}'.format(min=g_minNum, max=g_maxNum)  
  
  #--------------- For .csv -----------------------------------------------------------------------
  _uploadfile(drive_service
              , fileName=LOTTO_DATA_FILENAME
              , mimeType='text/csv')

  #--------------- For .json -----------------------------------------------------------------------
  _uploadfile(drive_service
              , fileName=LOTTO_DATA_FILENAME
              , mimeType='application/json')
  
  #--------------- For .gsheet -------------------------------------------
  _uploadfile(drive_service
              , fileName=LOTTO_DATA_FILENAME
              , mimeType='application/vnd.google-apps.spreadsheet')


def _uploadfile(service, fileName, mimeType):
  searchfile_metadata = {
      'name': fileName,
      'mimeType': mimeType,
      'parents': '1WXbbz1f0QEdltlsYEFpqt_21zm-vkDQq' #Colab Notebooks
  }
  print(searchfile_metadata, '\n')
  items = searchFromDrive(service, searchfile_metadata)
  print(items, '\n')

  #When the file in the specific folder, use 'parents':["parent id"]
  uploadfile_metadata = {
      'name': fileName,
      'mimeType': mimeType,
      'parents': [items[0].get('parents') if items else '1WXbbz1f0QEdltlsYEFpqt_21zm-vkDQq']   #Colab Notebooks
  }
  __uploadfile(service, uploadfile_metadata, items[0] if items else None )
  

def __uploadfile(service, meta_data, file):
  #file = None

  # Get a file for updating an existed file
  # if file_id:  
  #   file = service.files().get(fileId=file_id, fields='id')

  filelocation=''
  #filelocation='{name}.csv'.format(name=meta_data['name'])    

  if meta_data['mimeType'] == 'text/csv' or  meta_data['mimeType'] =='application/vnd.google-apps.spreadsheet':
    filelocation='{name}.csv'.format(name=meta_data['name'])    
  elif meta_data['mimeType']=='application/json':
    filelocation='{name}.json'.format(name=meta_data['name'])

  print('Guessed type - ', mimetypes.guess_type(filelocation)[0])

  media = MediaFileUpload( filelocation, #'content/drive/MyDrive/Colab Notebooks/lottoWinInfo.csv',
                        mimetype= mimetypes.guess_type(filelocation)[0],#meta_data['mimeType'],
                        resumable=True)

  print(meta_data)
  if file is None:
    file = service.files().create( 
                                    body=meta_data
                                    , media_body=media
                                    , fields='id, name, parents, mimeType' 
                                  ).execute()
    print('Created {2} {3} File ID: {0}, {1}'.format(file.get('id'), file.get('parents')
    , file.get('mimeType'), file.get('name')))
  else:
    file = service.files().update(
                                    fileId=file.get('id')
                                    , media_body=media
                                    , fields='id, name, parents, mimeType'
                                    ).execute()
    print('Updated {2} {3} File ID: {0}, {1}'.format(file.get('id'), file.get('parents')
    , file.get('mimeType'), file.get('name')))

  print('==============================================================================')

'''
@meta_data
EXAMPLE: folder_metadata = {
      'name': "Colab Notebooks",
      'mimeType': 'application/vnd.google-apps.folder',
      'parents': '0AJWOtiRiz6tlUk9PVA'
  }
'''
def searchFromDrive(service, meta_data):  
  page_token = None  
  while True:
      print("Searching.....")
      response = service.files().list(q="name='{name}' and mimeType='{mimeType}' and parents='{parents}'"
                                          .format(
                                              name=meta_data['name']
                                              , mimeType=meta_data['mimeType']
                                              , parents=meta_data['parents']
                                              )
                                          , spaces='drive'
                                          , fields='nextPageToken, files(id, name, mimeType, parents)'
                                          , pageToken=page_token).execute()
      
      for file in response.get('files', []):
          # Process change
          print ('Found file: %s (%s) - type:%s parents:%s ' % (file.get('name'), file.get('id'), file.get('mimeType'), file.get('parents')))
      page_token = response.get('nextPageToken', None)
      if page_token is None:
          break

  return response.get('files', [])

In [ ]:
from pandas.io.parsers.readers import read_csv
from typing_extensions import Literal
import mimetypes
from google.colab import data_table
data_table.enable_dataframe_formatter()

#from google.colab import drive
from google.colab import files
import sys, os

from tqdm import tqdm

def mount(path):
  from google.colab import drive
  
  mntPath = path#'/content/drive'
  if not os.path.ismount(mntPath):
    drive.mount(mntPath)
  else:
    print("Aleady Mounted at ", mntPath)

def updateLottoData(_min=1, _max=getMaxRoundNum()):   
  mount('/content/drive')

  global g_minNum, g_maxNum
  g_minNum=_min
  g_maxNum=_max
  targetFileName = "./lottoWinInfo_{min}_{max}".format(min=_min,max=_max)

  jsonfilename = '{filename}.json'.format(filename=targetFileName)
  csvfilename = '{filename}.csv'.format(filename=targetFileName)

  print(jsonfilename, csvfilename)

  if not (os.path.isfile(csvfilename) and os.path.isfile(jsonfilename)):
    df = getLottoWinInfo(g_minNum, g_maxNum)
    df.to_csv(csvfilename)
    df.to_json(jsonfilename)
    print('\n', df, '\n')   
  
  DownloadtoComputer(jsonfilename)
  DownloadtoComputer(csvfilename)
  
  uploadDataFilesToMyDrive(targetFileName)


def DownloadtoComputer(fileName):  

  if os.path.isfile(fileName):
    files.download(fileName)


updateLottoData(1000)




Aleady Mounted at  /content/drive
./lottoWinInfo_1000_1005.json ./lottoWinInfo_1000_1005.csv


100%|██████████| 6/6 [00:06<00:00,  1.05s/it]


    RoundNo        Date  Num1  Num2  Num3  Num4  Num5  Num6  bnsNum
1     1000  2022-01-29     2     8    19    22    32    42      39
2     1001  2022-02-05     6    10    12    14    20    42      15
3     1002  2022-02-12    17    25    33    35    38    45      15
4     1003  2022-02-19     1     4    29    39    43    45      31
5     1004  2022-02-26     7    15    30    37    39    44      18
6     1005  2022-03-05     8    13    18    24    27    29      17 



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Authenticating...
{'name': 'lottoWinInfo_1000_1005', 'mimeType': 'text/csv', 'parents': '1WXbbz1f0QEdltlsYEFpqt_21zm-vkDQq'} 

Searching.....
Found file: lottoWinInfo_1000_1005 (1bfBe5YJbGfqc7h0c3sZdNVxp447dxKI8) - type:text/csv parents:['1WXbbz1f0QEdltlsYEFpqt_21zm-vkDQq'] 
[{'id': '1bfBe5YJbGfqc7h0c3sZdNVxp447dxKI8', 'name': 'lottoWinInfo_1000_1005', 'mimeType': 'text/csv', 'parents': ['1WXbbz1f0QEdltlsYEFpqt_21zm-vkDQq']}] 

Guessed type -  text/csv
{'name': 'lottoWinInfo_1000_1005', 'mimeType': 'text/csv', 'parents': [['1WXbbz1f0QEdltlsYEFpqt_21zm-vkDQq']]}
Updated text/csv lottoWinInfo_1000_1005 File ID: 1bfBe5YJbGfqc7h0c3sZdNVxp447dxKI8, ['1WXbbz1f0QEdltlsYEFpqt_21zm-vkDQq']
{'name': 'lottoWinInfo_1000_1005', 'mimeType': 'application/json', 'parents': '1WXbbz1f0QEdltlsYEFpqt_21zm-vkDQq'} 

Searching.....
Found file: lottoWinInfo_1000_1005 (1WyMQyvfbSe0BfKNDj50yUZNpen4uq9Ym) - type:application/json parents:['1WXbbz1f0QEdltlsYEFpqt_21zm-vkDQq'] 
[{'id': '1WyMQyvfbSe0BfKNDj50yUZNpen

In [ ]:
#!pip install import-ipynb
#import import_ipynb

#!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, files
from oauth2client.client import GoogleCredentials
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.discovery import build

import os, io, sys

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

'''
mimeType
.py : 'application/x-python-code'
.ipynb : 'application/vnd.google.colaboratory'
'''

#module = drive.CreateFile({'title':'Test01.ipynb', 'mimeType':'application/vnd.google.colaboratory', 'parents':[{"kind": "drive#fileLink","id": '1WXbbz1f0QEdltlsYEFpqt_21zm-vkDQq'}] })
module = drive.CreateFile({'title':'Test01.py', 'mimeType':'application/x-python-code', 'parents':[{"kind": "drive#fileLink","id": '1WXbbz1f0QEdltlsYEFpqt_21zm-vkDQq'}] })
print(module)


#module.GetContentFile('drive/MyDrive/Colab Notebooks/Test01.py')



drive_service = build('drive', 'v3')

request = drive_service.files().get_media(fileId='1KbYO6ibXzCiMLQbSzu278oD2mpxP3IHd')
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))



# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile({'q': "'1WXbbz1f0QEdltlsYEFpqt_21zm-vkDQq' in parents"}).GetList()

for f in file_list:
  print('title: %s, id: %s, mimetype: %s' % (f['title'], f['id'],f['mimeType']))

import pandas as pd
# from Test01 import TestFunc

open('Test.py', 'wb')

import sys
#sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks')



#----------------------------------------------------------------------
my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/MyDrive/Colab Notebooks', my_path)
sys.path.insert(0, my_path)
#---------------------------------------------------------------------------









In [ ]:
import os, sys

my_path = '/content/notebooks'
# # Colab Notebooks 안에 my_env 폴더에 패키지 저장
# os.symlink('/content/drive/MyDrive/Colab Notebooks', my_path)
sys.path.insert(0, my_path)



In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, files
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

'''
<mimeType>
.py : 'application/x-python-code'
.ipynb : 'application/vnd.google.colaboratory'

<id>
root
Colab Notebooks: 1WXbbz1f0QEdltlsYEFpqt_21zm-vkDQq
'''

module = drive.CreateFile({'title':'customModuleController.py', 'mimeType':'application/x-python-code', 'parents':[{"kind": "drive#fileLink","id": '1WXbbz1f0QEdltlsYEFpqt_21zm-vkDQq'}] })
print(module)
module.Upload()

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile({'q': "'1WXbbz1f0QEdltlsYEFpqt_21zm-vkDQq' in parents"}).GetList()
# file_list = drive.ListFile({'q': "'root' in parents"}).GetList()

for f in file_list:
  print('title: %s, id: %s, mimetype: %s' % (f['title'], f['id'],f['mimeType']))

GoogleDriveFile({'title': 'customModuleController.py', 'mimeType': 'application/x-python-code', 'parents': [{'kind': 'drive#fileLink', 'id': '1WXbbz1f0QEdltlsYEFpqt_21zm-vkDQq'}]})
title: customModuleController.py, id: 1l-EGZjKDvUKdnbtPXSlIN4Q93hlO4FWK, mimetype: application/x-python-code
title: customModuleController.py, id: 1gKqzsSdfyfj8tHFyPCGa2cz7r0c8zpWh, mimetype: application/x-python-code
title: decodeLottoInfo.ipynb, id: 1k-XC5FVfE2-PeYYDZzz7LY9QKz2Pc_SU, mimetype: application/vnd.google.colaboratory
title: lottoDataController.py, id: 1ASM2gy3azKoq8_vQPizvt90C1fEg6SE2, mimetype: application/x-python-code
title: lottoDataController.py, id: 1indJx0QnnGWnmkwuaOKpKWly_zIC1MIl, mimetype: application/x-python-code
title: lottoDataController.py, id: 1peSzdABDC72rofFjq5D3uVyaHYHG7k33, mimetype: application/x-python-code
title: customModuleController.py, id: 1g9hYe16g57206z6VjJ6nh8Olyfa5tbWi, mimetype: application/x-python-code
title: lottoDataController.py, id: 1KbYO6ibXzCiMLQbSzu278oD

In [ ]:
#Restart Runtime Notebook
import os
os.kill(os.getpid(), 9)

In [ ]:
import os, sys

my_path = '/content/notebooks'
# # Colab Notebooks 안에 my_env 폴더에 패키지 저장
# #os.symlink('/content/drive/MyDrive/Colab Notebooks', my_path)
sys.path.insert(0, my_path)
os.path.join('/content/drive', '/content/drive/MyDrive/Colab Notebooks')

'/content/drive/MyDrive/Colab Notebooks'

In [ ]:
from customModuleController import CustomModules as myModules

m = myModules()
m.createFile('lottoDataController.py', 'application/x-python-code')

# if open('/content/notebooks/lottoDataController.py', 'rb'):
#   print('| am here!!!')
# else:
#   print('I am not here!!!')
#myModules().listModules()

| am already here222!!!
